<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Lying_about_your_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lying about your Age

![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144) ![](https://deliterate.wordpress.com/wp-content/uploads/2013/11/58452819_adventures_of_pinocchio_answer_1_xlarge.jpeg?w=144)

Annie, Betty, Carrie, Darla, and Eve recently found out that all of their birthdays were on the same day, though they are different ages. On their mutual birthday, they were jabbering away, flapping their gums about their recent discovery. And, lucky me, I was there. Some of the things that I overheard were...

1. Darla said to Betty: "I'm nine years older than Eve."
2. Eve said to Betty: "I'm seven years older than Annie."
3. Annie said to Betty: "Your age is exactly 70% greater than mine."
4.  Betty said to Carrie: "Eve is younger than you."
5.  Carrie said to Darla: "The difference between our ages is six years."
6.  Carrie said to Annie: "I'm ten years older than you."
7.  Carrie said to Annie: "Betty is younger than Darla."
8. Betty said to Carrie: "The difference between your age and Darla's is the same as the difference between Darla's and Eve's."

Since I knew these people -- and how old they were, I knew that they were not telling the whole truth. After thinking about it, I realized that when one of them spoke to somebody older than herself, everything she said was true, but when speaking to somebody younger, everything she said was false.

How old is each person?

In [23]:
!pip install ortools
from ortools.sat.python import cp_model

In [24]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, x,y):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__x = x
        self.__y = y
        self.__solution_count = 0


    def on_solution_callback(self):
        self.__solution_count += 1

        print(f'Solution {self.__solution_count}')
        print([(i,self.Value(self.__x[i])) for i in self.__x])
        print([(i,self.Value(self.__y[i])) for i in self.__y])
        print('\n')

    def solution_count(self):
        return self.__solution_count

In [53]:
def SearchForAllSolutionsSampleSat():
  # Creates the model and set solver
  model = cp_model.CpModel()
  solver = cp_model.CpSolver()

  max_age = 100
  num_constraints = 8

  people = ['Annie', 'Betty', 'Carrie', 'Darla', 'Eve' ]

  x = {i:model.NewIntVar(1,max_age,f"x_{i}") for i in people}

  y = {i:model.NewBoolVar(f"y_{i}") for i in range(1,num_constraints+1)}

  # All different ages
  model.AddAllDifferent([x[i] for i in x.keys()])

  # 1. Darla said to Betty: "I'm nine years older than Eve."
  model.Add(x['Darla'] < x['Betty']).OnlyEnforceIf(y[1])
  model.Add(x['Darla'] > x['Betty']).OnlyEnforceIf(y[1].Not())
  model.Add(x['Darla'] == x['Eve'] + 9).OnlyEnforceIf(y[1])
  model.Add(x['Darla'] != x['Eve'] + 9).OnlyEnforceIf(y[1].Not())

  # 2. Eve said to Betty: "I'm seven years older than Annie."
  model.Add(x['Eve'] < x['Betty']).OnlyEnforceIf(y[2])
  model.Add(x['Eve'] > x['Betty']).OnlyEnforceIf(y[2].Not())
  model.Add(x['Eve'] == x['Annie'] + 7).OnlyEnforceIf(y[2])
  model.Add(x['Eve'] != x['Annie'] + 7).OnlyEnforceIf(y[2].Not())

  # 3. Annie said to Betty: "Your age is exactly 70% greater than mine."
  model.Add(x['Annie'] < x['Betty']).OnlyEnforceIf(y[3])
  model.Add(x['Annie'] > x['Betty']).OnlyEnforceIf(y[3].Not())
  model.Add(10*x['Betty'] == 17*x['Annie']).OnlyEnforceIf(y[3])
  model.Add(10*x['Betty'] != 17*x['Annie']).OnlyEnforceIf(y[3].Not())

  # 4.  Betty said to Carrie: "Eve is younger than you."
  model.Add(x['Betty'] < x['Carrie']).OnlyEnforceIf(y[4])
  model.Add(x['Betty'] > x['Carrie']).OnlyEnforceIf(y[4].Not())
  model.Add(x['Eve'] < x['Carrie']).OnlyEnforceIf(y[4])
  model.Add(x['Eve'] > x['Carrie']).OnlyEnforceIf(y[4].Not())

  # 5.  Carrie said to Darla: "The difference between our ages is six years."
  model.Add(x['Carrie'] < x['Darla']).OnlyEnforceIf(y[5])
  model.Add(x['Carrie'] > x['Darla']).OnlyEnforceIf(y[5].Not())
  model.Add(x['Darla'] - x['Carrie'] == 6).OnlyEnforceIf(y[5])
  model.Add(x['Darla'] - x['Carrie'] != 6).OnlyEnforceIf(y[5].Not())

  # 6.  Carrie said to Annie: "I'm ten years older than you."
  model.Add(x['Carrie'] < x['Annie']).OnlyEnforceIf(y[6])
  model.Add(x['Carrie'] > x['Annie']).OnlyEnforceIf(y[6].Not())
  model.Add(x['Carrie'] == x['Annie'] + 10).OnlyEnforceIf(y[6])
  model.Add(x['Carrie'] != x['Annie'] + 10).OnlyEnforceIf(y[6].Not())

  # 7.  Carrie said to Annie: "Betty is younger than Darla."
  model.Add(x['Carrie'] < x['Annie']).OnlyEnforceIf(y[7])
  model.Add(x['Carrie'] > x['Annie']).OnlyEnforceIf(y[7].Not())
  model.Add(x['Betty'] < x['Darla']).OnlyEnforceIf(y[7])
  model.Add(x['Betty'] > x['Darla']).OnlyEnforceIf(y[7].Not())

  # 8. Betty said to Carrie: "The difference between your age and Darla's is the same as the difference between Darla's and Eve's."
  model.Add(x['Betty'] < x['Carrie']).OnlyEnforceIf(y[8])
  model.Add(x['Betty'] > x['Carrie']).OnlyEnforceIf(y[8].Not())
  model.Add(x['Carrie'] - x['Darla'] == x['Darla'] - x['Eve']).OnlyEnforceIf(y[8])
  model.Add(x['Carrie'] - x['Darla'] != x['Darla'] - x['Eve']).OnlyEnforceIf(y[8].Not())

  solution_printer = VarArraySolutionPrinter(x,y)

  # Enumerate all solutions.
  solver.parameters.enumerate_all_solutions = True

  status = solver.Solve(model, solution_printer)

  print(f"Status = {solver.StatusName(status)}")

SearchForAllSolutionsSampleSat()

Solution 1
[('Annie', 30), ('Betty', 51), ('Carrie', 55), ('Darla', 46), ('Eve', 37)]
[(1, 1), (2, 1), (3, 1), (4, 1), (5, 0), (6, 0), (7, 0), (8, 1)]


Status = OPTIMAL
